Exercise 1: Assume we have $N$ individual and J products in a market. $X_{ij}$ is the characteristic of product j for individual i. $J_i$ is the choice of individual i among the products. 

To generate the data, I assume $u_{ij} = \beta_0 + \beta_1X_j + \epsilon_{ij}$ where $X_j$ is created as independent standard normal random variables, $\epsilon_{ij}$ follows a T1EV, $\beta_0 = 5$ and $\beta_1 = 2$.

In [33]:
# See the below link for Distribution
# https://juliastats.github.io/Distributions.jl/stable/starting/

Pkg.add("Distributions")
using LinearAlgebra
using Random, Distributions


 Resolving package versions...
  Updating `~/Project.toml`
 [no changes]
  Updating `~/Manifest.toml`
 [no changes]


In [37]:
# Parameters
J = 5 #Number of products
N = 300 #Number of individuals
T = 200 #Grid size
Scaler = 100 #Scaler
β_0 = 1
β_1 = 1
d = Gumbel() # or we can use d = Normal()
# Variables
Random.seed!(123) #Setting the seed
X = [ones(J)  rand(d, J)]
δ = [β_0 β_1]*X'
Random.seed!(7) #Setting the seed
ϵ = rand(d, N , J)
u = δ .+ ϵ

300×5 Array{Float64,2}:
 3.90574  2.79003  2.00856   1.43804    0.554162 
 3.19416  4.74665  0.800837  1.29867    0.264042 
 2.67082  6.3724   5.06831   1.49991   -0.0421086
 1.40145  3.52802  2.3419    2.97349    3.68593  
 2.94666  3.22067  2.04728   1.9406     1.20417  
 2.03065  3.76649  1.9791    2.25355    4.14641  
 1.84724  2.9471   2.89981   1.22447    1.29086  
 1.83928  6.58527  3.036     2.55685    0.550866 
 4.10114  2.99565  2.71211   0.112594   2.23358  
 3.21944  4.31196  1.40082   0.901441   0.352027 
 2.42526  3.84837  1.35632   0.405072   1.60445  
 3.12666  5.83045  1.87242   2.56852    0.461131 
 2.66725  6.79904  1.4101    2.22404    0.350511 
 ⋮                                               
 2.40875  4.11995  2.87331   0.658679   0.250351 
 2.77753  4.47289  1.82827   0.264549   1.48889  
 2.67437  3.61717  0.242352  2.50728    3.25639  
 3.51111  3.47687  0.979932  2.31273    3.08166  
 3.17361  6.81555  2.1185    2.82601    1.30986  
 1.5734   3.06504  1.32803

In [38]:
u_max, ind = findmax(u, dims = 2)
ind
choice_mat = (u .>= u_max)
s = sum( choice_mat, dims = 1)/N

1×5 Array{Float64,2}:
 0.166667  0.653333  0.09  0.04  0.05

The probability that individual i chooses product j is
$$
P( i~chooses~j) = \frac{exp(\delta_j)}{\Sigma_{k \in J}~exp(\delta_k)}
$$
Therefore, the maximum likelihood estimator is
$$
L = \Pi_{i=1}^N\Pi_{j=1}^J~P(i->j)^{1(J_i == j)}
$$
and the log-likelihood function is
$$
logL = \Sigma_{i=1}^N\Sigma_{j=1}^J~1(J_i == j)\{log(P(i->j))\}
$$

In [39]:
L = zeros(T-1 , T-1)
for b_0 = 1:T-1
    for b_1 = 1:T-1
        P_nom = exp.( [b_0/Scaler  b_1/Scaler]*X' )
        P_sum = sum( P_nom )
        P = P_nom/ P_sum
        L[b_0,b_1] = sum( choice_mat * P')
    end
end
findmax(L)

(183.54579335032867, CartesianIndex(1, 199))

I have to compute the log likelihood function for each value of b0 and b1 which means I should multiply choice_mat with probs and take the summation and keep the max. Then repeat the whole process 100 times to find the average b0 and b1 the its variance.